**Ваша любимая домашка**

Кто проходил курс GPU прекрасно знают датасет.
Данные находятся в файле Train_Set_90621.csv
Amount Defaulted - эту переменную нужно удалить=)

Что ожидается? - творчество)

    1) Начните с анализа баланса классов, пропусков, статистик при помощи DataFrame API
    2) Посомтрите статистики, заполните пропуски при помощи уже MLlib
    3) Соберите пайплайн, похожий на наш, где будет обработка данных, обучение моделей и все при помощи Spark
    4) Разбейте данные на train/test + реализуйте подбор параметров одним из способов спарка
    5) Cохраниет пайплайн на диск
    6) Проверьте качество модели на отложенной test выборке

### Срок выполнения - 02.03.2023


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import StringIndexer, IndexToString, OneHotEncoder
from pyspark.ml.feature import Imputer, VectorAssembler, StandardScaler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from google.colab import drive
drive.mount('/content/drive')
spark = SparkSession.builder.appName("Homework_Pipeline").getOrCreate()

Mounted at /content/drive


In [ ]:
df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Train_Set_90621.csv", header=True, inferSchema=True)

In [ ]:
df.printSchema()

root
 |-- Application ID: integer (nullable = true)
 |-- Bank Masked: string (nullable = true)
 |-- Bank Type: string (nullable = true)
 |-- Approved_Timestamp: string (nullable = true)
 |-- Name Masked: string (nullable = true)
 |-- Business Owner State: string (nullable = true)
 |-- Business_Industry_Type_Code: integer (nullable = true)
 |-- Approved_Year: integer (nullable = true)
 |-- New Business: integer (nullable = true)
 |-- Term: integer (nullable = true)
 |-- BankState: string (nullable = true)
 |-- Interest Rate: integer (nullable = true)
 |-- Employees: integer (nullable = true)
 |-- Gross Disbursed Amount: integer (nullable = true)
 |-- Term_years: integer (nullable = true)
 |-- Jobs Retained: integer (nullable = true)
 |-- Male to Female Employees Ratio: integer (nullable = true)
 |-- Expected Company Income: long (nullable = true)
 |-- Funds available with company: integer (nullable = true)
 |-- Gross_Apprv_Amount: integer (nullable = true)
 |-- Company Branch Code: inte

In [ ]:
df.show(5)

+--------------+--------------+---------+------------------+--------------------+--------------------+---------------------------+-------------+------------+----+-------------+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+----------------+-------------------+--------------+
|Application ID|   Bank Masked|Bank Type|Approved_Timestamp|         Name Masked|Business Owner State|Business_Industry_Type_Code|Approved_Year|New Business|Term|    BankState|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Carry-forward Credit|Documents Provided|Balance Left|Amount Defaulted|Final_Appved_Amount|Defa

In [ ]:
target='Default_Status'

In [ ]:
df.groupBy(target).count().show()


+--------------+------+
|Default_Status| count|
+--------------+------+
|             1| 35238|
|             0|159880|
+--------------+------+



In [ ]:
missing_counts = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns])
missing_counts.show()


+--------------+-----------+---------+------------------+-----------+--------------------+---------------------------+-------------+------------+----+---------+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+----------------+-------------------+--------------+
|Application ID|Bank Masked|Bank Type|Approved_Timestamp|Name Masked|Business Owner State|Business_Industry_Type_Code|Approved_Year|New Business|Term|BankState|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Carry-forward Credit|Documents Provided|Balance Left|Amount Defaulted|Final_Appved_Amount|Default_Status|
+--------------+----

In [ ]:
df.describe().show()

+-------+-----------------+---------------+---------+------------------+-------------------+--------------------+---------------------------+-----------------+-------------------+------------------+-----------+------------------+------------------+----------------------+-----------------+------------------+------------------------------+-----------------------+----------------------------+--------------------+-------------------+------------------+-----------------+--------------------+------------------+----------------+-----------------+--------------------+-------------------+
|summary|   Application ID|    Bank Masked|Bank Type|Approved_Timestamp|        Name Masked|Business Owner State|Business_Industry_Type_Code|    Approved_Year|       New Business|              Term|  BankState|     Interest Rate|         Employees|Gross Disbursed Amount|       Term_years|     Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|  Gross_Apprv_Amount|

In [ ]:
df = df.withColumn("Documents Provided", F.when(F.col("Documents Provided") == "Y", 1).otherwise(0))

In [ ]:
df = df.withColumn("Carry-forward Credit", F.when(F.col("Carry-forward Credit") == "Y", 1).otherwise(0))

In [ ]:
df = df.withColumn("Bank Type", F.when(F.col("Bank Type") == "Govt", 1).otherwise(0))

In [ ]:
df = df.withColumn("Expected Company Income", df["Expected Company Income"].cast(DoubleType()))

In [ ]:
numeric_cols = [field.name for field in df.schema.fields
                if field.dataType.simpleString() in ["int", "double", "long", "float"] and field.name not in ['Application ID', 'Approved_Year']]

In [ ]:
cat_cols = [field.name for field in df.schema.fields
                if field.name not in numeric_cols and field.name not in ['Approved_Timestamp', 'Bank Masked', 'Name Masked', 'Application ID']]
#отфильтровываем дату, имя клиента и имя банка, тк слишком много уникальных значений, чтобы выделить категории

In [ ]:
df[numeric_cols].show(5)

+---------+---------------------------+------------+----+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+----------------+-------------------+--------------+
|Bank Type|Business_Industry_Type_Code|New Business|Term|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Carry-forward Credit|Documents Provided|Balance Left|Amount Defaulted|Final_Appved_Amount|Default_Status|
+---------+---------------------------+------------+----+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+-------------

In [ ]:
df[cat_cols].show(5)

+--------------------+-------------+-------------+
|Business Owner State|Approved_Year|    BankState|
+--------------------+-------------+-------------+
|       Uttar Pradesh|         2005|Uttar Pradesh|
|       Uttar Pradesh|         2006|Uttar Pradesh|
|       Uttar Pradesh|         2007|  Maharashtra|
|              Kerala|         2009|        Delhi|
|               Delhi|         2002|  Maharashtra|
+--------------------+-------------+-------------+
only showing top 5 rows



In [ ]:
#df = df.withColumn("Business Owner State", df["Business Owner State"].cast(DoubleType()))
owner_indexer = StringIndexer(inputCol="Business Owner State", outputCol="Business_Owner_State_indexer")
owner_indexer = owner_indexer.fit(df)
df = owner_indexer.transform(df)

In [ ]:
df.show(5)

+--------------+--------------+---------+------------------+--------------------+--------------------+---------------------------+-------------+------------+----+-------------+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+----------------+-------------------+--------------+----------------------------+
|Application ID|   Bank Masked|Bank Type|Approved_Timestamp|         Name Masked|Business Owner State|Business_Industry_Type_Code|Approved_Year|New Business|Term|    BankState|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Carry-forward Credit|Documents Provided|Balance Left|Amount Defau

In [ ]:
bank_indexer = StringIndexer(inputCol="BankState", outputCol="BankStateindexer")
bank_indexer = bank_indexer.fit(df)
df = bank_indexer.transform(df)

In [ ]:
df.show()

+--------------+---------------+---------+------------------+--------------------+--------------------+---------------------------+-------------+------------+----+-------------+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+----------------+-------------------+--------------+----------------------------+----------------+
|Application ID|    Bank Masked|Bank Type|Approved_Timestamp|         Name Masked|Business Owner State|Business_Industry_Type_Code|Approved_Year|New Business|Term|    BankState|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Carry-forward Credit|Documents Provided|Balanc

In [ ]:
ohe_encoder = OneHotEncoder(inputCols=["Business_Owner_State_indexer", "BankStateindexer"],
                        outputCols=["Business_Owner_State_vector", "BankStateVector"])
ohe_encoder = ohe_encoder.fit(df)
df = ohe_encoder.transform(df)

In [ ]:
df.show(5)

+--------------+--------------+---------+------------------+--------------------+--------------------+---------------------------+-------------+------------+----+-------------+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+----------------+-------------------+--------------+----------------------------+----------------+---------------------------+---------------+
|Application ID|   Bank Masked|Bank Type|Approved_Timestamp|         Name Masked|Business Owner State|Business_Industry_Type_Code|Approved_Year|New Business|Term|    BankState|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|Company Branch Code|City or Rural|Jobs Generated|Carr

In [ ]:
df.count()

195118

In [ ]:
df.select(F.col('Bank Masked')).distinct().count()

42530

In [ ]:
df.select(F.col('Name Masked')).groupBy('Name Masked').count().show()

+--------------------+-----+
|         Name Masked|count|
+--------------------+-----+
|Business_Name_140550|    1|
|Business_Name_139609|    1|
| Business_Name_73472|    1|
| Business_Name_62132|    2|
| Business_Name_28654|    1|
|Business_Name_105970|    1|
| Business_Name_84125|    2|
| Business_Name_94290|    1|
|Business_Name_104886|    1|
| Business_Name_70810|    2|
|Business_Name_119181|    1|
|Business_Name_153633|    1|
|Business_Name_108035|    1|
| Business_Name_77717|    2|
| Business_Name_97956|    1|
| Business_Name_13852|    1|
|  Business_Name_4697|    2|
| Business_Name_50851|    2|
| Business_Name_30859|    3|
|Business_Name_104949|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
df.select(F.col('Business Owner State')).groupBy('Business Owner State').count().show()

+--------------------+-----+
|Business Owner State|count|
+--------------------+-----+
|           Karnataka|  798|
|              Kerala|  543|
|          Tamil Nadu| 2040|
|              Punjab|  616|
|                NULL|    3|
|             Haryana|56421|
|              Orissa| 2603|
|               Delhi|58546|
|         Maharashtra|48088|
|       Uttar Pradesh|24167|
|         Uttarakhand| 1293|
+--------------------+-----+



In [ ]:
df.select(F.col('Bank Type')).groupBy('Bank Type').count().show()

+---------+------+
|Bank Type| count|
+---------+------+
|        1| 64888|
|        0|130230|
+---------+------+



In [ ]:
numeric_cols.remove('Company Branch Code')

In [ ]:
impute_cols = [c for c in numeric_cols if c != target]

imputer = Imputer(inputCols=impute_cols,
                  outputCols=[c + "_imputed" for c in impute_cols])
df = imputer.fit(df).transform(df)

In [ ]:
df.show(10)

+--------------+---------------+---------+------------------+--------------------+--------------------+---------------------------+-------------+------------+----+-------------+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------------+-------------+--------------+--------------------+------------------+------------+----------------+-------------------+--------------+----------------------------+----------------+---------------------------+---------------+-----------------+-----------------------------------+--------------------+------------+---------------------+-----------------+------------------------------+------------------+---------------------+--------------------------------------+-------------------------------+------------------------------------+--------------------------+---------------------+----------------------+------------------------

In [ ]:
df[numeric_cols].show(10)

+---------+---------------------------+------------+----+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------+--------------+--------------------+------------------+------------+----------------+-------------------+--------------+
|Bank Type|Business_Industry_Type_Code|New Business|Term|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|City or Rural|Jobs Generated|Carry-forward Credit|Documents Provided|Balance Left|Amount Defaulted|Final_Appved_Amount|Default_Status|
+---------+---------------------------+------------+----+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------+--------------+-----

In [ ]:
for c in impute_cols:
    df = df.drop(c).withColumnRenamed(c + "_imputed", c)

### ML Pipeline

In [ ]:
feature_cols = [c for c in numeric_cols if c != target]


In [ ]:
feature_cols.append('Business_Owner_State_indexer')
feature_cols.append('BankStateindexer')

In [ ]:
df[feature_cols].show(10)

+---------+---------------------------+------------+----+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+----------------------------+------------------+-------------+--------------+--------------------+------------------+------------+----------------+-------------------+----------------------------+----------------+
|Bank Type|Business_Industry_Type_Code|New Business|Term|Interest Rate|Employees|Gross Disbursed Amount|Term_years|Jobs Retained|Male to Female Employees Ratio|Expected Company Income|Funds available with company|Gross_Apprv_Amount|City or Rural|Jobs Generated|Carry-forward Credit|Documents Provided|Balance Left|Amount Defaulted|Final_Appved_Amount|Business_Owner_State_indexer|BankStateindexer|
+---------+---------------------------+------------+----+-------------+---------+----------------------+----------+-------------+------------------------------+-----------------------+--------------------

In [ ]:
ohe_encoder = OneHotEncoder(inputCols=["Business_Owner_State_indexer", "BankStateindexer"],
                        outputCols=["Business_Owner_State_Vector", "BankStateVec"])

In [ ]:
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

In [ ]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

In [ ]:
lr = LogisticRegression(featuresCol="scaledFeatures", labelCol=target)

In [ ]:
df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Train_Set_90621.csv", header=True, inferSchema=True)

In [ ]:
df = df.withColumn("Expected Company Income", df["Expected Company Income"].cast(DoubleType()))
df = df.withColumn("Business Owner State", df["Business Owner State"].cast(DoubleType()))
df = df.withColumn("BankState", df["BankState"].cast(DoubleType()))
df = df.withColumn("Documents Provided", F.when(F.col("Documents Provided") == "Y", 1).otherwise(0))
df = df.withColumn("Carry-forward Credit", F.when(F.col("Carry-forward Credit") == "Y", 1).otherwise(0))
df = df.withColumn("Bank Type", F.when(F.col("Bank Type") == "Govt", 1).otherwise(0))
owner_indexer = StringIndexer(inputCol="Business Owner State", outputCol="Business_Owner_State_indexer")
bank_indexer = StringIndexer(inputCol="BankState", outputCol="BankStateindexer")


In [ ]:
df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Train_Set_90621.csv", header=True, inferSchema=True)


# Explicitly rename columns to remove spaces and special characters
for c in df.columns:
    df = df.withColumnRenamed(c, c.replace(' ', '_').replace('-', '_'))

# Fill remaining missing numeric values with 0
numeric_columns = ['Business_Industry_Type_Code', 'New_Business', 'Term', 'Interest_Rate',
                   'Employees', 'Gross_Disbursed_Amount', 'Jobs_Retained',
                   'Male_to_Female_Employees_Ratio', 'Expected_Company_Income',
                   'Funds_available_with_company', 'Gross_Apprv_Amount',
                   'City_or_Rural', 'Jobs_Generated', 'Balance_Left',
                   'Amount_Defaulted', 'Final_Appved_Amount']

df = df.fillna(0, subset=numeric_columns)

categorical_columns = ['Bank_Type', 'Business_Owner_State', 'Carry_forward_Credit', 'Documents_Provided']
df = df.fillna('Unknown', subset=categorical_columns)

indexers = [StringIndexer(inputCol=col, outputCol=f"{col}_Indexed", handleInvalid="keep") for col in categorical_columns]

for col_name in numeric_columns:
    df = df.withColumn(col_name, F.col(col_name).cast("double"))

assembler_input = [f"{col}_Indexed" for col in categorical_columns] + numeric_columns
assembler = VectorAssembler(inputCols=assembler_input, outputCol="features", handleInvalid="keep")

lr = LogisticRegression(featuresCol="features", labelCol="Default_Status")

pipeline = Pipeline(stages=indexers + [assembler, lr])

pipeline_model = pipeline.fit(df)


+--------------------+--------------+----------+
|            features|Default_Status|prediction|
+--------------------+--------------+----------+
|[0.0,3.0,0.0,0.0,...|             0|       0.0|
|[1.0,3.0,1.0,0.0,...|             1|       0.0|
|[0.0,3.0,1.0,0.0,...|             0|       0.0|
|[1.0,9.0,0.0,0.0,...|             0|       0.0|
|[1.0,0.0,0.0,1.0,...|             0|       0.0|
|[0.0,2.0,0.0,1.0,...|             0|       0.0|
|(20,[3,4,5,6,7,8,...|             0|       0.0|
|[0.0,3.0,0.0,0.0,...|             0|       0.0|
|[1.0,1.0,0.0,1.0,...|             0|       0.0|
|(20,[1,4,5,6,7,8,...|             0|       0.0|
+--------------------+--------------+----------+
only showing top 10 rows



In [ ]:
target='Default_Status'

In [ ]:
pipeline = Pipeline(stages=[owner_indexer, bank_indexer, ohe_encoder, assembler, scaler, lr])

In [ ]:
transformed_df = pipeline_model.transform(df)

In [ ]:
train_df, test_df = df.randomSplit([0.7, 0.3], seed=42)

In [ ]:
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

In [ ]:
evaluator = BinaryClassificationEvaluator(labelCol=target)

In [ ]:
cv = CrossValidator(estimator=pipeline,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator,
                    numFolds=3)

In [ ]:
cvModel = cv.fit(train_df)

In [ ]:
cvModel.bestModel.write().overwrite().save("saved_pipeline")

predictions = cvModel.transform(test_df)
predictions.select("features", "Default_Status", "prediction", "probability").show(5)

# Compute and print AUC
auc = evaluator.evaluate(predictions)
print("Test AUC: ", auc)

+--------------------+--------------+----------+--------------------+
|            features|Default_Status|prediction|         probability|
+--------------------+--------------+----------+--------------------+
|[0.0,3.0,1.0,0.0,...|             0|       0.0|[0.82195747651187...|
|(20,[3,4,5,6,7,8,...|             0|       0.0|[0.82195747651187...|
|[1.0,1.0,0.0,1.0,...|             0|       0.0|[0.82195747651187...|
|(20,[1,4,5,6,7,8,...|             0|       0.0|[0.82195747651187...|
|[0.0,2.0,0.0,0.0,...|             0|       0.0|[0.82195747651187...|
+--------------------+--------------+----------+--------------------+
only showing top 5 rows

Test AUC:  0.8267301783002465


In [ ]:
spark.stop()
